In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint


#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

C:\Users\R_zia\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\R_zia\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\R_zia\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\R_zia\anaconda3\envs\PythonData\lib\site-packages

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
#Drop the non-beneficial ID columns, 'EIN' and 'NAME', 'SPECIAL_CONSIDERATIONS'

application_df= application_df.drop(["EIN", "NAME", "SPECIAL_CONSIDERATIONS", "CLASSIFICATION"], axis=1)
application_df=application_df.dropna()
#application_df=application_df[application_df["INCOME_AMT"] > "0"]
application_df.head()

,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,ProductDev,Association,1,0,5000,1
1,T3,Independent,Preservation,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,ProductDev,Association,1,0,5000,0
3,T3,CompanySponsored,Preservation,Trust,1,10000-24999,6692,1
4,T3,Independent,Heathcare,Trust,1,100000-499999,142590,1


In [3]:
#APPLICATION COUNT 
application_counts= application_df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are less than ...?

replace_application = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Generate our categorical variable lists
application_cat=application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE', 'AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT']

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
#  YOUR CODE GOES HERE
encode_df=pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Merge one-hot encoded features and drop the originals

application_df=application_df.merge(encode_df, left_index=True, right_index=True)
application_df=application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Split our preprocessed data into our features and target arrays

X=application_df.drop(["IS_SUCCESSFUL"],1).values
y=application_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 200
hidden_nodes_layer2 = 100
hidden_nodes_layer3= 50

nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               7200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 32,401
Trainable params: 32,401
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Compile the model

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=5,
    save_weights_only=True,
    save_freq=1000)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
# Train the model

fit_model=nn.fit(X_train, y_train, epochs=200, callbacks=[cp_callback])

Epoch 1/200
  512/25724 [..............................] - ETA: 23s - loss: 0.7082 - acc: 0.4980 
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 1824/25724 [=>............................] - ETA: 11s - loss: 0.6954 - acc: 0.5307
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 2592/25724 [==>...........................] - ETA: 8s - loss: 0.6948 - acc: 0.5285 
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 4000/25724 [===>..........................] - ETA: 6s - loss: 0.6931 - acc: 0.5340
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 4960/25724 [====>.........................] - ETA: 5s - loss: 0.6927 - acc: 0.5361
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 5984/25724 [=====>........................] - ETA: 4s - loss: 0.6924 - acc: 0.5368
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 6816/25724 [======>.......................] - ETA: 4s - loss: 0.6927 - acc: 0.5335
Epoch 00001: saving model to checkpoints/weights.01.hd

 6528/25724 [======>.......................] - ETA: 2s - loss: 0.6937 - acc: 0.5149
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 7776/25724 [========>.....................] - ETA: 2s - loss: 0.6937 - acc: 0.5135
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 8704/25724 [=========>....................] - ETA: 2s - loss: 0.6932 - acc: 0.5163
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 9568/25724 [==========>...................] - ETA: 2s - loss: 0.6924 - acc: 0.5209
Epoch 00003: saving model to checkpoints/weights.03.hdf5
10688/25724 [===========>..................] - ETA: 2s - loss: 0.6922 - acc: 0.5237
Epoch 00003: saving model to checkpoints/weights.03.hdf5
11968/25724 [============>.................] - ETA: 1s - loss: 0.6924 - acc: 0.5233
Epoch 00003: saving model to checkpoints/weights.03.hdf5
12896/25724 [==============>...............] - ETA: 1s - loss: 0.6924 - acc: 0.5224
Epoch 00003: saving model to checkpoints/weights.03.hdf5
13792/25724 [

13440/25724 [==============>...............] - ETA: 1s - loss: 0.6925 - acc: 0.5269
Epoch 00005: saving model to checkpoints/weights.05.hdf5
14784/25724 [================>.............] - ETA: 1s - loss: 0.6924 - acc: 0.5277
Epoch 00005: saving model to checkpoints/weights.05.hdf5
15808/25724 [=================>............] - ETA: 1s - loss: 0.6924 - acc: 0.5280
Epoch 00005: saving model to checkpoints/weights.05.hdf5
16416/25724 [==================>...........] - ETA: 1s - loss: 0.6924 - acc: 0.5281
Epoch 00005: saving model to checkpoints/weights.05.hdf5
17376/25724 [===================>..........] - ETA: 1s - loss: 0.6921 - acc: 0.5297
Epoch 00005: saving model to checkpoints/weights.05.hdf5
18720/25724 [====================>.........] - ETA: 0s - loss: 0.6924 - acc: 0.5274
Epoch 00005: saving model to checkpoints/weights.05.hdf5
19840/25724 [======================>.......] - ETA: 0s - loss: 0.6924 - acc: 0.5285
Epoch 00005: saving model to checkpoints/weights.05.hdf5
20576/25724 [

Epoch 00007: saving model to checkpoints/weights.07.hdf5
20640/25724 [=======================>......] - ETA: 0s - loss: 0.6925 - acc: 0.5240
Epoch 00007: saving model to checkpoints/weights.07.hdf5
21600/25724 [========================>.....] - ETA: 0s - loss: 0.6922 - acc: 0.5258
Epoch 00007: saving model to checkpoints/weights.07.hdf5
22560/25724 [=========================>....] - ETA: 0s - loss: 0.6922 - acc: 0.5261
Epoch 00007: saving model to checkpoints/weights.07.hdf5
23648/25724 [==========================>...] - ETA: 0s - loss: 0.6923 - acc: 0.5260
Epoch 00007: saving model to checkpoints/weights.07.hdf5
24576/25724 [===========================>..] - ETA: 0s - loss: 0.6923 - acc: 0.5266
Epoch 00007: saving model to checkpoints/weights.07.hdf5
25724/25724 [==============================] - 4s 144us/sample - loss: 0.6923 - acc: 0.5278
Epoch 8/200
   32/25724 [..............................] - ETA: 7s - loss: 0.6932 - acc: 0.5312
Epoch 00008: saving model to checkpoints/weights.0

25724/25724 [==============================] - 4s 137us/sample - loss: 0.6920 - acc: 0.5276
Epoch 10/200
  512/25724 [..............................] - ETA: 2s - loss: 0.6895 - acc: 0.5547
Epoch 00010: saving model to checkpoints/weights.10.hdf5
 1696/25724 [>.............................] - ETA: 3s - loss: 0.6920 - acc: 0.5224
Epoch 00010: saving model to checkpoints/weights.10.hdf5
 2816/25724 [==>...........................] - ETA: 3s - loss: 0.6923 - acc: 0.5210
Epoch 00010: saving model to checkpoints/weights.10.hdf5
 3872/25724 [===>..........................] - ETA: 3s - loss: 0.6922 - acc: 0.5235
Epoch 00010: saving model to checkpoints/weights.10.hdf5
 4640/25724 [====>.........................] - ETA: 3s - loss: 0.6921 - acc: 0.5254
Epoch 00010: saving model to checkpoints/weights.10.hdf5
 5664/25724 [=====>........................] - ETA: 3s - loss: 0.6922 - acc: 0.5258
Epoch 00010: saving model to checkpoints/weights.10.hdf5
 6944/25724 [=======>......................] - ET

...

16736/25724 [==================>...........] - ETA: 1s - loss: 0.6923 - acc: 0.5280
Epoch 00190: saving model to checkpoints/weights.190.hdf5
17440/25724 [===================>..........] - ETA: 1s - loss: 0.6923 - acc: 0.5283
Epoch 00190: saving model to checkpoints/weights.190.hdf5
18688/25724 [====================>.........] - ETA: 0s - loss: 0.6924 - acc: 0.5271
Epoch 00190: saving model to checkpoints/weights.190.hdf5
19808/25724 [======================>.......] - ETA: 0s - loss: 0.6925 - acc: 0.5261
Epoch 00190: saving model to checkpoints/weights.190.hdf5
20704/25724 [=======================>......] - ETA: 0s - loss: 0.6924 - acc: 0.5270
Epoch 00190: saving model to checkpoints/weights.190.hdf5
21856/25724 [========================>.....] - ETA: 0s - loss: 0.6921 - acc: 0.5284
Epoch 00190: saving model to checkpoints/weights.190.hdf5
22528/25724 [=========================>....] - ETA: 0s - loss: 0.6922 - acc: 0.5282
Epoch 00190: saving model to checkpoints/weights.190.hdf5
23456/

22144/25724 [========================>.....] - ETA: 0s - loss: 0.6917 - acc: 0.5312
Epoch 00192: saving model to checkpoints/weights.192.hdf5
23168/25724 [==========================>...] - ETA: 0s - loss: 0.6919 - acc: 0.5301
Epoch 00192: saving model to checkpoints/weights.192.hdf5
24544/25724 [===========================>..] - ETA: 0s - loss: 0.6920 - acc: 0.5289
Epoch 00192: saving model to checkpoints/weights.192.hdf5
25376/25724 [============================>.] - ETA: 0s - loss: 0.6920 - acc: 0.5288
Epoch 00192: saving model to checkpoints/weights.192.hdf5
25724/25724 [==============================] - 3s 115us/sample - loss: 0.6920 - acc: 0.5291
Epoch 193/200
  608/25724 [..............................] - ETA: 2s - loss: 0.6922 - acc: 0.5362
Epoch 00193: saving model to checkpoints/weights.193.hdf5
 1696/25724 [>.............................] - ETA: 3s - loss: 0.6921 - acc: 0.5165
Epoch 00193: saving model to checkpoints/weights.193.hdf5
 2624/25724 [==>..........................

 1664/25724 [>.............................] - ETA: 2s - loss: 0.6927 - acc: 0.5264
Epoch 00195: saving model to checkpoints/weights.195.hdf5
 2528/25724 [=>............................] - ETA: 2s - loss: 0.6932 - acc: 0.5162
Epoch 00195: saving model to checkpoints/weights.195.hdf5
 3424/25724 [==>...........................] - ETA: 2s - loss: 0.6930 - acc: 0.5190
Epoch 00195: saving model to checkpoints/weights.195.hdf5
 4832/25724 [====>.........................] - ETA: 2s - loss: 0.6927 - acc: 0.5226
Epoch 00195: saving model to checkpoints/weights.195.hdf5
 5760/25724 [=====>........................] - ETA: 2s - loss: 0.6925 - acc: 0.5240
Epoch 00195: saving model to checkpoints/weights.195.hdf5
 6624/25724 [======>.......................] - ETA: 2s - loss: 0.6921 - acc: 0.5272
Epoch 00195: saving model to checkpoints/weights.195.hdf5
 7840/25724 [========>.....................] - ETA: 2s - loss: 0.6922 - acc: 0.5273
Epoch 00195: saving model to checkpoints/weights.195.hdf5
 8512/

 7232/25724 [=======>......................] - ETA: 2s - loss: 0.6931 - acc: 0.5221
Epoch 00197: saving model to checkpoints/weights.197.hdf5
 8224/25724 [========>.....................] - ETA: 2s - loss: 0.6929 - acc: 0.5236
Epoch 00197: saving model to checkpoints/weights.197.hdf5
 9664/25724 [==========>...................] - ETA: 1s - loss: 0.6925 - acc: 0.5259
Epoch 00197: saving model to checkpoints/weights.197.hdf5
10560/25724 [===========>..................] - ETA: 1s - loss: 0.6927 - acc: 0.5248
Epoch 00197: saving model to checkpoints/weights.197.hdf5
11392/25724 [============>.................] - ETA: 1s - loss: 0.6925 - acc: 0.5255
Epoch 00197: saving model to checkpoints/weights.197.hdf5
12768/25724 [=============>................] - ETA: 1s - loss: 0.6922 - acc: 0.5284
Epoch 00197: saving model to checkpoints/weights.197.hdf5
13664/25724 [==============>...............] - ETA: 1s - loss: 0.6923 - acc: 0.5261
Epoch 00197: saving model to checkpoints/weights.197.hdf5
14624/

13536/25724 [==============>...............] - ETA: 1s - loss: 0.6917 - acc: 0.5280
Epoch 00199: saving model to checkpoints/weights.199.hdf5
14400/25724 [===============>..............] - ETA: 1s - loss: 0.6918 - acc: 0.5276
Epoch 00199: saving model to checkpoints/weights.199.hdf5
15200/25724 [================>.............] - ETA: 1s - loss: 0.6919 - acc: 0.5271
Epoch 00199: saving model to checkpoints/weights.199.hdf5
16096/25724 [=================>............] - ETA: 1s - loss: 0.6918 - acc: 0.5278
Epoch 00199: saving model to checkpoints/weights.199.hdf5
17536/25724 [===================>..........] - ETA: 0s - loss: 0.6917 - acc: 0.5290
Epoch 00199: saving model to checkpoints/weights.199.hdf5
18240/25724 [====================>.........] - ETA: 0s - loss: 0.6915 - acc: 0.5301
Epoch 00199: saving model to checkpoints/weights.199.hdf5
19232/25724 [=====================>........] - ETA: 0s - loss: 0.6914 - acc: 0.5310
Epoch 00199: saving model to checkpoints/weights.199.hdf5
20672/

In [12]:
#Export our model to HDF5 file
nn.save("AlphabetSoupCharity_OPTIMIZATION2.h5")

In [13]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('AlphabetSoupCharity_OPTIMIZATION2.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.7423 - acc: 0.4943
Loss: 0.7422724546148896, Accuracy: 0.494344025850296
